In [ ]:
SCANVI_ID: str = '256_30_Level1_run1'
BUCKET_DIRPATH: str
REF_NAME: str = "MAIN"

In [10]:
import scarches as sca
import scvi
scvi.settings.dl_num_workers = 0
import pickle as pkl
import scvi
import scanpy as sc
import os
import numpy as np
from dotenv import load_dotenv
from lightning.pytorch.loggers import WandbLogger

 captum (see https://github.com/pytorch/captum).


In [ ]:
assert load_dotenv()

In [ ]:
def here(fpath):
    return os.path.join(BUCKET_DIRPATH, fpath)

In [ ]:
overwriteData = True

In [ ]:
# Load the h5ad file
adata_path = '04_MAIN_geneUniverse.h5ad'
reference_adata = sc.read_h5ad(adata_path)

In [ ]:
if 'SPLIT' in REF_NAME:
    fold_idx = int(REF_NAME.split('_')[1])
    print(fold_idx)
    with open(here('03_Downstream_Analysis/PatientClassifier/5foldCV/data/K_FOLD_cellID.pkl'), 'rb') as f:
        splits = pkl.load(f)
    train_idx, _ = splits[fold_idx]
    reference_adata = reference_adata[train_idx].copy()

In [ ]:
reference_adata

In [ ]:
scvi_model = scvi.model.SCVI.load(here(f"03_Downstream_Analysis/PatientClassifier/scANVI/results/01_reference/scANVI_{SCANVI_ID}"), adata=reference_adata)
scanvi_model = scvi.model.SCANVI.from_scvi_model(scvi_model, unlabeled_category = "Unknown")

In [ ]:
scvi_parameter_dict = {}
trainer_kwargs = dict(
    checkpointing_monitor = 'elbo_validation',
    early_stopping_monitor = 'reconstruction_loss_validation',
    early_stopping_patience = 2,
    early_stopping = True,
    max_epochs = 100,
    batch_size = 128
)

plan_kwargs = dict(
    lr = 1e-4,
    #reduce_lr_on_plateau = True
)

scvi_parameter_dict.update(trainer_kwargs)
scvi_parameter_dict.update(plan_kwargs)

In [ ]:
logger = WandbLogger(
    project='inflammation_atlas_PatientClassifier_scANVI', 
    entity='inflammation',
    config=scvi_parameter_dict,
    name = f'scANVI_finetuning_{REF_NAME}_{SCANVI_ID}',
)

In [ ]:
scanvi_model.train(
    logger=logger,
    plan_kwargs=plan_kwargs,
    **trainer_kwargs
)

In [ ]:
if overwriteData:
    scanvi_model.save(
        here(f"03_Downstream_Analysis/PatientClassifier/scANVI/results/012_fine_tuning/models/scANVI_{SCANVI_ID}_finetuning"), 
        overwrite = True,
        save_anndata = False)